# Event-driven machine learning for PdM focusing on log preprocessing

An advanced data-driven predictive maintenance approach is presented in [10].
The objective of this research work is to develop an alerting system that provides
early notifications to aviation engineers for upcoming aircraft failures, providing
the needed time for the maintenance actions. The aviation is a well-documented
field, as all the maintenance and 
flight data are systematically logged. Hence,
**event-based techniques** can leverage this special characteristic and **provide effective predictive solutions**. The **main challenge** is to **cope** with the large set of **og
entries** that are essentially irrelevant **to the main failures**.
In [10], the emphasis is placed on **log preprocessing**; therefore, we will refer
to this methodology as **LPPdM**.

10.Korvesis, P., Besseau, S., Vazirgiannis, M.: Predictive maintenance in aviation:
Failure prediction from post 
ight reports. In: IEEE Int. Conf. on Data Engineering
(ICDE). pp. 1414{1422 (2018)

### Setup

In [270]:
#Make the necessary imports.

suppressMessages(library(CORElearn))
suppressMessages(library(dplyr))
suppressMessages(library(plyr))
suppressMessages(library(data.table))
suppressMessages(library(randomForest))
suppressMessages(library(xgboost))
suppressMessages(library(ggplot2))
suppressMessages(library(grid))
suppressMessages(library(argparser))
suppressMessages(library(stringr))
suppressMessages(library(keras))
suppressMessages(library(kerasR))
suppressMessages(library(plsdepot))
suppressMessages(library(class))
suppressMessages(library(FNN))
suppressMessages(library(Rfast))

### Init variables

In [343]:
# Make an argument parser named p and keep there the necessary variables.
p <- arg_parser("Implementation of the AIRBUS Predictor")

# Add a positional argument
p <- add_argument(p, "id", help="experiment ID")
p <- add_argument(p, "train", help="training dataset")
p <- add_argument(p, "test", help="test dataset")
p <- add_argument(p, "fet", help="different types of the fault events",default=151) #51#11  
p <- add_argument(p, "tet", help="type of the target fault events",default=151) #51#11
p <- add_argument(p, "--rre", help="remove rare events", default=TRUE)
p <- add_argument(p, "--rfe", help="remove frequent events", default=TRUE)
p <- add_argument(p, "--kofe", help="keep only first event", default=TRUE)
p <- add_argument(p, "--milt", help="MIL as written in the text of the paper", default=TRUE)
p <- add_argument(p, "--mili", help="MIL as shonw in the Figure of the paper", default=FALSE)
p <- add_argument(p, "--milthres", help="MIL threshold to the sigmoid function for over-sampling", default=0.4)
p <- add_argument(p, "--steepness", help="steepness of the sigmoid function", default=0.7)
p <- add_argument(p, "--midpoint", help="midpoint of the sigmoid function (in days)", default=151) #51
p <- add_argument(p, "--fs", help="apply feature selection", default=FALSE)
p <- add_argument(p, "--top", help="# of features to keep in feature selection", default=3)#we have max 10 feautures(before was 200)
p <- add_argument(p, "--rer", help="rare events ratio of the target event frequency", default=0.5)
p <- add_argument(p, "--fer", help="frequent events ratio of the frequency of the most frequent event", default=0.8)
p <- add_argument(p, "--milw", help="MIL window size (in days)", default=6)
p <- add_argument(p, "--pthres", help="prediction threshold to the Risk value for a true positive episode", default=0.5)
p <- add_argument(p, "--seed", help="seed for RF", default=400)
p <- add_argument(p, "--csv", help="output for csv", default=TRUE)

#p <- add_argument(p, "--step", help="feature selection decrease step", default=10)#5#1

p <- add_argument(p, "--spme", help="export datasets for sequential pattern minning", default=FALSE)

setwd("C:/Program Files (x86)")
p <- add_argument(p, "--java", help="the java path", default="./Java/jdk1.8.0_192/bin/java.exe")

setwd("C:/Users/petsi")
p <- add_argument(p, "--python", help="the python path", default="./Anaconda/python.exe")

setwd("C:")
p <- add_argument(p, "--cep", help="complex event processing path", default="C:/Users/petsi/Documents/ptyxiakh/my_spmrules.py")
p <- add_argument(p, "--spmf", help="the spmf path", default="C:/Users/petsi/Documents/ptyxiakh/spmf.jar")


p <- add_argument(p, "--conf", help="minimum support (minsup)", default="70%")
p <- add_argument(p, "--minti", help="minimum time interval allowed between two succesive itemsets of a sequential pattern", default=4)
p <- add_argument(p, "--maxti", help="maximum time interval allowed between two succesive itemsets of a sequential pattern", default=5)
p <- add_argument(p, "--minwi", help="minimum time interval allowed between the first itemset and the last itemset of a sequential pattern", default=4)
p <- add_argument(p, "--maxwi", help="maximum time interval allowed between the first itemset and the last itemset of a sequential pattern", default=5)

p <- add_argument(p, "--minwint", help="min # of days before failure to expect a warning for true positive decision", default=1)#2  (1)
p <- add_argument(p, "--maxwint", help="max # of days before failure to expect a warning for true positive decision", default=30)#5 (30)



#comparison with myatm
p <- add_argument(p, "--X", help="# of segments/sub-windows", default=5)#3#1     (6)
p <- add_argument(p, "--M", help="segment legth (in days)", default=2)#2#1       (5)   
p <- add_argument(p, "--Y", help="length of the prediction window (in days)", default=10)#2#3#2     (30)
p <- add_argument(p, "--Z", help="length of the buffer window (in days)", default=5)  #             (0)
p <- add_argument(p, "--N", help="moving step (in days)", default=10)#2#2#1                         (10)
p <- add_argument(p, "--step", help="feature selection decrease step", default=40)#5#1              (30)

### Define the necessary variables

In [344]:
# Define the necessary variables.

argv = data.frame() #make a data frame named argv
#if( length(commandArgs(trailingOnly = TRUE)) != 0){
if(FALSE){
  argv <- parse_args(p)
} else {
  #argv <- parse_args(p,c(1,"C:/Users/Public/ptyxiakh/training_my_dataset2.csv","C:/Users/Public/ptyxiakh/testing_my_dataset2.csv",11,11))
  #argv <- parse_args(p,c(1,"C:/Users/Public/ptyxiakh/training_my_dataset_5years.csv","C:/Users/Public/ptyxiakh/testing_my_dataset_5years.csv",51,51))
  argv <- parse_args(p,c(1,"C:/Users/petsi/Documents/ptyxiakh/training_my_dataset_150.csv",
                         "C:/Users/petsi/Documents/ptyxiakh/testing_my_dataset_150.csv",151,151))    
}


#init the variables
id = argv$id

train_path=argv$train
test_path=argv$test

b_length = argv$fet
target_event = argv$tet

target_event_frequency_proportion_rare = argv$rer
max_event_frequency_proportion_frequent = argv$fer

top_features = argv$top

milw = argv$milw
F_thres = argv$milthres

s = argv$steepness
midpoint = argv$midpoint
acceptance_threshold = argv$pthres

export_spm = argv$spme

max_warning_interval = argv$maxwint
min_warning_interval = argv$minwint

csv = argv$csv

seed = argv$seed

step=argv$step

#comparison with myatm
X = argv$X
M = argv$M
Y = argv$Y
Z = argv$Z
N = argv$N
#milw=X*M #like OW
milw=10
F_thres=0

midpoint=(X*M+Z+Y)#/2
#midpoint=10

acceptance_threshold = 0.7#0.7
s=0.7#0.9 (0.8)

### Reading function
**function: read_dataset**

In [345]:
# Function for reading the csv file and save it to a two column table.

read_dataset <- function(path){
  dataset = read.table(path, header = TRUE, sep = ",", dec = ".", comment.char = "#")
  dataset[, 2]  <- as.numeric(dataset[, 2])
  return(dataset)
}

### Read train and test set

The recorded log types read from csv files.
One csv file(at **train_path**) has the **training_set** and the other(at **test_path**) the **testing_set**.

In [346]:
# Reading train and test set.

training_set = read_dataset(train_path)
test_set =  read_dataset(test_path)

print("The test_set and training_set looks like:")
print(head(test_set))

[1] "The test_set and training_set looks like:"
  Timestamps Event_id
1 2016-12-31       36
2 2016-12-31       43
3 2016-12-31       58
4 2016-12-31      112
5 2016-12-31      120
6 2016-12-31      130


## Funtions for preprocessing

In order to increase the effectiveness of the approach standard **preprocessing** techniques are applied: 

- **Rare events (more rare than the target event)**, are considered as extremely rare, hence they are removed to reduce the dimensionality of the data. 

- **Most frequent events** usually do not contain significant information since they correspond to issues of minor importance. A tf-idf (term-frequency - inverted document frequency) or a simple threshold-based approach can be used to remove most frequent events. 

- **Multiple occurrences** of the same event in the same segment can either be noise or may not provide useful information. Hence, multiple occurrences are shrank into a single one. 

- Events of minor importance occur and appear in every segment until their underlying cause is treated by the technical experts. Hence, the **first occurrence of events** that occur in consecutive segments is maintained. 

- To deal with the imbalance of the labels (given that the target event is rare) and as several events appear shortly before the occurrence of the target event, but only a small subset of them is related to the target event, the authors use **Multiple Instance Learning (MIL) bagging the events**. A single bag contains fault events of a single day. Using MIL, the data closer to the target event (a threshold is specified), are over-sampled.

- A statistical **feature selection technique**, based on the distance of the fault events with the target event is applied, to filter out fault events, which are far from the target event.



**1) function: preprocess**

In [347]:
preprocess <- function(ds,TEST_DATA,REMOVE_RARE_EVENTS,REMOVE_FREQUENT_EVENTS,
                       KEEP_ONLY_FIRST_OCCURENESS,MULTI_INSTANCE_LEARNING_TEXT,
                       MULTI_INSTANCE_LEARNING_IMAGE,FEATURE_SELECTION,top_features,
                       s,midpoint,b_length,target_event,
                       target_event_frequency_proportion_rare,max_event_frequency_proportion_frequent,w,F_thres){
  #print(nrow(ds))
  #remove events that appear < n times. We consider n = (target event frequency)/2
  if(REMOVE_RARE_EVENTS){
    rre<-remove_rare_events(ds,target_event_frequency_proportion_rare,ret=TRUE) #function: remove_rare_events
  }
  #print(nrow(ds))
  #remove events that appear > n times. We consider n = (target event frequency)/2
  if(REMOVE_FREQUENT_EVENTS){
    rfe<-remove_frequent_events(ds,max_event_frequency_proportion_frequent,ret=TRUE) #function: remove_frequent_events
  }
  print(nrow(ds))
  #create for the dataset(ds) a dataframe keeping for each day the frequency of the fault events
  episodes_list = compute_frequency_list(ds,b_length)
  #return(    episodes_list)
  if(REMOVE_RARE_EVENTS){
    if(length(rre)>0){
      for(i in 1:length(rre)){
        col=paste("e_",rre[i],sep = "")
        episodes_list[[col]]=c(1:nrow(episodes_list))*0
      }
    }
  }
  if(REMOVE_FREQUENT_EVENTS){
    if(length(rfe)>0){
      for(i in 1:length(rfe)){
        col=paste("e_",rfe[i],sep = "")
        episodes_list[[col]]=c(1:nrow(episodes_list))*0
      }
    }
  }
  #print((episodes_list[1,]))
  
  print("The starting episodes_list is:")
  #print(head((episodes_list)[[1]]))
  print("------------------------------------------------------------------------------------------------------")
  
  #binarize the vector
  #for(ep_index in (1:length(episodes_list))){
  #ep = episodes_list[[ep_index]]
  #ep[2:(ncol(ep)-1)][ep[2:(ncol(ep)-1)] > 0] = 1
  #episodes_list[[ep_index]] = ep
  episodes_list[2:(ncol(episodes_list)-1)][episodes_list[2:(ncol(episodes_list)-1)] > 0] = 1
  #print(length(episodes_list[,1]))
  #}
  print("After binirizing:")
  #print(head((episodes_list)[[1]]))
  print("------------------------------------------------------------------------------------------------------")  
  
  
  #keep only the first occurness of event in consecutive segments
  if(KEEP_ONLY_FIRST_OCCURENESS){
    episodes_list <- keep_only_first_occureness(episodes_list) #function: keep_only_first_occureness
    print("After first occurancing preprocess:")
    #print(head((episodes_list)[[1]]))
    print("------------------------------------------------------------------------------------------------------")  
  }
  
  
  
  #multi-instance learning to increase the pattern frequency
  if(MULTI_INSTANCE_LEARNING_TEXT){
    episodes_list <- mil_text(w,F_thres,episodes_list,b_length) #function: mil_text
    print("After MIL text (the returning list):")
    #print(head((episodes_list)[[1]]))
    print("------------------------------------------------------------------------------------------------------")  
  } else if(MULTI_INSTANCE_LEARNING_IMAGE){
    episodes_list <- mil_image(w,F_thres,episodes_list,b_length) #function: mil_image
    print("After MIL image (the returning list):")
    #print(head((episodes_list)[[1]]))
    print("------------------------------------------------------------------------------------------------------")  
  }
  
  
  
  return(episodes_list)
}

### 2) Functions for removing the less useful data

**2a) function: remove_rare_events**

**2b) function: remove_frequent_events**

**2c) function: keep_only_first_occureness**

In [348]:
# Functions for removing events.

#Function for removing the rare events, considering target event's frequency.
remove_rare_events <- function(ds,target_event_frequency_proportion_rare,
                              ret=FALSE){
  if(!csv){
    print("~~~~~~~APPLYING PROPREPROCESSING: REMOVE RARE EVENTS~~~~~~~")
  }
  a = table(ds$Event_id) #table that shows the total appearances per day

  target_event_frequency = a[names(a)==target_event] #total appearances of target event

  #find the rear events :the events that their frequency is smaller than a threashold
  #threashold           :percentage of total appearances of target event
  #percentage           :given by the user usually near 0.5 (target_event_frequency_proportion_rare) 
  rare_events = as.integer(names(a[a < target_event_frequency*target_event_frequency_proportion_rare])) 
  
  print("The removing rare events(columns) are:")
  print(rare_events)  
  print("------------------------------------------------------------------------------------------------------")  
  
  if(ret){
    return(rare_events)
  }  
    
  return(ds[!(ds$Event_id %in% rare_events),]) #return the dataset without the rear events
}

#Function for removing the frequent events, considering the maximum frequency that observed.
remove_frequent_events <- function(ds,max_event_frequency_proportion_frequent,
                                  ret=FALSE){
  if(!csv){
    print("~~~~~~~APPLYING PROPREPROCESSING: REMOVE FREQUENT EVENTS~~~~~~~")
  }
  a = table(ds$Event_id) #table that shows the total appearances per day

  max_freq = sort(a,decreasing = TRUE)[[1]] #maximum total appearances of a target event

  #find the rear events :the events that their frequency is bigger than a threashold
  #threashold           :percentage of the maximum total appearances of a target event
  #percentage           :given by the user usually near 0.5 (max_event_frequency_proportion_frequent)  
  frequent_events = as.integer(names(a[a > max_freq*max_event_frequency_proportion_frequent]))

  print("The removing frequent_events events(columns) are:")
  print(frequent_events)  
  print("------------------------------------------------------------------------------------------------------")
  
  if(ret){
    return(frequent_events)
  }    
    
  return(ds[!(ds$Event_id %in% frequent_events),])
}

#Function for keeping only first occurances of consecutively events.
keep_only_first_occureness <- function(episodes_list){
  if(!csv){
    print("~~~~~~~APPLYING PROPREPROCESSING: KEEP ONLY FIRST OCCURENESS~~~~~~~")
  }
  #for every episode in the episodes_list
  
  #keep the episode  
  ep = episodes_list
  
  #For every segment of each episode starting from the end up to the second segment. 
  #We need to keep only the 1st occurness of consequtive events, hence starting from the end is the easy way.
  for(i in (nrow(ep):2)){
    #as we deal with binary vectors, to find the indeces that both vectors have "1" we sum them and check for "2"s in the result
    matches = which((ep[i,2:length(ep)]+ep[i-1,2:length(ep)]) %in% c(2))
    
    #replace the 1s with 0s in the matching positions of the segment that is closer to the end of the episode
    ep[i,2:length(ep)][c(matches)] = 0
  }
  #save changes to the episodes_list
  episodes_list = ep
  
  return(episodes_list) #return the new episodes_list
}

### 3) Functions to compute episodes list

The post flight logs are partitioned in ranges
defined by the occurrences of the fault(**episodes**) that PdM targets. These ranges are further
partitioned into time-segments, which may correspond to a day or to a single
usage of the equipment(a **day** for our example). The idea is that the segments that are closer to the end
of the range may contain fault events that are potentially indicative of the main
event. The **goal** is to **learn a function** that quantifies the **risk** of the targeted
failure occurring in the near future, given the events that precede it. Hence, a
**sigmoid function** is proposed, which **maps higher values to the segments that are
closer to the target event**. The **steepness** and **shift** of the sigmoid function are
configured to better map the expectation of the time before the target event at
which correlated events will start occurring.



### Episode explanation

For the **test_set** the episodes are presented

In [349]:
#Function for creating frequency vectors for each day.
compute_frequency_list <- function(ds2years,b_length){
  
  #data.frame for frequencies
  episode_df <- data.frame(Timestamps=as.Date(character()),Event_id=integer())
  
  #Change ds2years(table) to episode_df(data frame)    
  
  #iterate over every line of the original dataset
  for(i in 1:nrow(ds2years)) {
    #get the current row of ds2years(table of data set)
    meas <- ds2years[i,]
    #add it to data frame  
    episode_df <- rbind(episode_df,data.frame(Timestamps=meas$Timestamps, Event_id=meas$Event_id))
    
  }
  #group by day
  aggr_episode_df = aggregate(episode_df[ ,2], FUN=function(x){return(x)}, by=list(as.Date(episode_df$Timestamps, "%Y-%m-%d")))
  
  #binarize the frequncy vector(function: compute_frequency_vectors)
  frequency_day_vectors = compute_frequency_vector(aggr_episode_df,b_length)
  
  return(frequency_day_vectors)
}

#Convert event vectors to binary vectors
compute_frequency_vector <- function(aggr_episode_df,b_length){
  
  #data frame for binary frequency vectors  
  freq_aggr_episode_df <- data.frame(matrix(ncol = b_length+1, nrow = 0))
  
  #x keeps the names of the columns. |Timestamps||e_1||e_2|...|e_b_length|  
  x <- c(c("Timestamps"), c(paste("e_",c(1:b_length),sep = "")))
  
  #iterate over every line(day) of the aggr_episode_df
  for(i in 1:nrow(aggr_episode_df)) {
    
    #init a vector with b_length zeros
    freq_vector = as.vector(integer(b_length))
    
    #get the current row of aggr_episode_df(frequency vector-data frame of data set)
    seg <- aggr_episode_df[i,]
    
    #for every value(fault event) in the current line(that happened in the current day)
    for(value in seg$x[[1]]){
      #replace the 0 in freq_vector with 1 at "value=fault event" position 
      freq_vector[[value]] = length(which(seg$x[[1]] == value))
    }
    
    #add a new line to the bin_aggr_epissode_df
    #we use a matrix holding the elements of the new_data.frame as matrix is able to store variable of different data types
    
    date = as.Date(seg$Group.1[[1]],origin = "1970-01-01")
    freq_vector=as.Date(freq_vector,origin = "1970-01-01")
    new_df = data.frame(matrix(c(date, freq_vector),nrow=1,ncol=b_length+1))
    freq_aggr_episode_df <- rbind(freq_aggr_episode_df,new_df)
  }
  #set column's name as x defines
  colnames(freq_aggr_episode_df) <- x
  
  #set column "Timestamps" x to a Date: "Y-m-d" column  
  freq_aggr_episode_df$Timestamps <- as.Date(freq_aggr_episode_df$Timestamps , origin="1970-01-01")
  
  return(freq_aggr_episode_df)
}

**3a) function: create_episodes_list**

**3b) function: compute_frequency_vectors**

**3c) function: compute_F**

In [350]:
#Functions for computing episodes list.

#Function for creating episodes list of each day's frequency vectors.
create_episodes_list <- function(ds,target_event,b_length,s,midpoint){
  if(!csv){
    print("~~~~~~~CREATING FREQUENCY VECTORS AND BINARIZE THEM~~~~~~~")
  }
  #devide in episodes
  target_event_spotted = FALSE
  
  #a list with data.frames for the episodes (each episode one data.frame)
  episodes_list = list()
  
  #data.frame for episodes
  episode_df <- data.frame(Timestamps=as.POSIXct(character()),Event_id=integer())
  
  #iterate over every line of the original dataset
  for(i in 1:nrow(ds)) {
    #get the current row of the ds
    meas <- ds[i,]
    #if it is the target event enable the appropriate flag
    if((meas$Event_id == target_event) || i==1 ){
      target_event_spotted = TRUE
    }
    
    #fill the episode data.frame with the events that are between two target events
    if(meas$Event_id != target_event && target_event_spotted){
      episode_df <- rbind(episode_df,data.frame(Timestamps=meas$Timestamps, Event_id=meas$Event_id))
    } else if(meas$Event_id == target_event && target_event_spotted && is.data.frame(episode_df) && nrow(episode_df) != 0){
      #a second occurness of the target event is spotted, close the episode
      
      #aggregate by day all the events to form the segments inside the episodes
      aggr_episode_df = aggregate(episode_df[ ,2], FUN=function(x){return(x)}, by=list(Timeframe=cut(as.POSIXct(episode_df$Timestamps, format="%Y-%m-%d"),"day"))) #%Y-%m-%dT%H:%M:%OSZ
      
      #binarize the frequncy vector
      bin_aggr_episode_df = compute_frequency_vectors(aggr_episode_df,b_length,s,midpoint) #function: compute_frequency_vectors
      
      #add the episode to the episodes_list
      episodes_list[[length(episodes_list)+1]] = bin_aggr_episode_df
      
      #reset episode_df to en empty data.frame
      episode_df <- data.frame(Timestamps=as.POSIXct(character()),Event_id=integer())
    }
  }
  return(episodes_list)
}

#Convert event vectors to binary vectors
compute_frequency_vectors <- function(aggr_episode_df,b_length,s,midpoint){
  
  #data frame for binary frequency vectors   
  freq_aggr_episode_df <- data.frame(matrix(ncol = b_length+2, nrow = 0))
  
  #x keeps the names of the columns. |Timestamps||e_1||e_2|...|Risk_F|   
  x <- c(c("Timestamps"), c(paste("e_",c(1:b_length),sep = "")), c("Risk_F"))
  
  
  for(i in 1:nrow(aggr_episode_df)) {
    
    #init a vector with b_length zeros
    freq_vector = as.vector(integer(b_length))
    
    #get the current row of aggr_episode_df(frequency vector-data frame of data set)  
    seg <- aggr_episode_df[i,]
    
    #for every value(fault event) in the current line(that happened in the current day)
    for(value in seg$x[[1]]){
      #replace the 0 in freq_vector with 1 at "value=fault event" position   
      freq_vector[[value]] = length(which(seg$x[[1]] == value))
    }
    #add a new line to the bin_aggr_epissode_df
    #we use a matrix holding the elements of the new_data.frame as matrix is able to store variable of different data types
    
    #F = compute_F(s,midpoint,i-1,nrow(aggr_episode_df)) #compute risk F (function: compute_F)
    F = compute_F(s,midpoint,i,nrow(aggr_episode_df)) #compute risk F (function: compute_F)  
    date = seg$Timeframe[[1]]
    new_df = data.frame(matrix(c(date, freq_vector,F),nrow=1,ncol=b_length+2))
    freq_aggr_episode_df <- rbind(freq_aggr_episode_df,new_df)
  }
  #set column's name as x defines  
  colnames(freq_aggr_episode_df) <- x
  
  return(freq_aggr_episode_df)
}

#The Risk function(sigmoid)
compute_F <- function(s,midpoint,t,ep_length){
  return(1/(1+exp(s*(ep_length-midpoint-t+1))))            

}

### 4) Multi Instance Learning functions

**4a) function: mil_text**

**4b) function: mil_image**

**4c) function: mil_text_atm**

**4d) function: my_mil**

In [351]:
#Function used for over_sampled the data near target_event
mil_text <- function(milw,F_thres,episodes_list,b_length){
  if(!csv){
    print("~~~~~~~APPLYING PROPREPROCESSING: MULTI INSTANCE LEARNING~~~~~~~")
  }
  window_df = data.frame(matrix(ncol = b_length+2, nrow = 0))
  
  #for every episode in the episodes_list
  for(ep_index in (1:length(episodes_list))){
    ep = episodes_list[[ep_index]]
    new_ep = data.frame(matrix(ncol = b_length+2, nrow = 0))
    i = 1
    while(i <= nrow(ep)){
      new_ep = rbind(new_ep,ep[i,])
      if(ep[i,][b_length+2] >= F_thres && nrow(window_df) < milw){
        window_df = rbind(window_df,ep[i,])
      }
      if(nrow(window_df) == milw || i == nrow(ep)){
        mean = colMeans(window_df)
        mean[mean > 0] = 1
        mf = data.frame(as.list(mean))
        mf[1] = ep[i,][1]
        mf[b_length+2] = ep[i,][b_length+2]
        #colnames(mf) = colnames(new_ep)
        new_ep = rbind(new_ep,mf)
        if(nrow(window_df) > 1){
          i = i - (nrow(window_df)-2)
        }
        window_df = data.frame(matrix(ncol = b_length+2, nrow = 0))
      }
      i = i + 1
    }
    episodes_list[[ep_index]] = new_ep
  }
  return(episodes_list)
}

#Function used for over_sampled the data near target_event
mil_image <- function(milw,F_thres,episodes_list,b_length){
  if(!csv){
    print("~~~~~~~APPLYING PROPREPROCESSING: MULTI INSTANCE LEARNING~~~~~~~")
  }
  
  #for every episode in the episodes_list
  for(ep_index in (1:length(episodes_list))){
    ep = episodes_list[[ep_index]]
    new_ep = data.frame(matrix(ncol = b_length+2, nrow = 0))
    #a data.frame with the vectors that need to be averaged
    window_df = data.frame(matrix(ncol = b_length+2, nrow = 0))
    i = 1
    while(i <= nrow(ep)){
      #new_ep = rbind(new_ep,ep[i,])
      if(nrow(window_df) < milw){
        window_df = rbind(window_df,ep[i,])
      }
      if(nrow(window_df) == milw || i == nrow(ep)){
        mean = colMeans(window_df)
        mean[mean > 0] = 1
        mf = data.frame(as.list(mean))
        mf[1] = ep[i,][1]
        mf[b_length+2] = ep[i,][b_length+2]
        #colnames(mf) = colnames(new_ep)
        new_ep = rbind(new_ep,mf)
        if(window_df[1,][b_length+2] >= F_thres && nrow(window_df) > 1){
          i = i - (nrow(window_df)-1)
        }
        window_df = data.frame(matrix(ncol = b_length+2, nrow = 0))
      }
      i = i + 1
    }
    episodes_list[[ep_index]] = new_ep
  }
  return(episodes_list)
}

#merged_episodes1=mil_text_atm(milw,0,episodes_list,b_length,N=N,Z=Z)
#Function used for over_sampled the data as atm does
mil_text_atm <- function(milw,F_thres,episodes_list,b_length,N=1,Z=0){
  if(!csv){
    print("~~~~~~~APPLYING PROPREPROCESSING: MULTI INSTANCE LEARNING~~~~~~~")
  }
  window_df = data.frame(matrix(ncol = b_length+2, nrow = 0))
  
  for(ep_index in (1:length(episodes_list))){
    
    ep = episodes_list[[ep_index]]
    
    new_ep = data.frame(matrix(ncol = b_length+2, nrow = 0))
    
    i = 1
    
    
    while(i <= nrow(ep)){
      if(i<=0){
        break
      }
      
      new_ep = rbind(new_ep,ep[i,])
      if(ep[i,][b_length+2] >= F_thres && nrow(window_df) < milw){
        window_df = rbind(window_df,ep[i,])
      }
      
      if(nrow(window_df) == milw || i == nrow(ep)-Z){  
        mean = colMeans(window_df)#mean of each column
        
        mean[mean > 0] = 1#binirize the mean of each column
        
        mf = data.frame(as.list(mean))#make list to frame
        
        mf[1] = ep[i,][1]#timestamp changed from 1 to i or row that are at the currend loop
        mf[b_length+2] = ep[i,][b_length+2]#RISK_F changed from 1 to the risk of the current i 
        
        new_ep = rbind(new_ep,mf)
        
        if(nrow(window_df) > 1){
          #i = i - (nrow(window_df)-2)#4-(4=+2) #moving step
          i = i - (nrow(window_df)-N) #moving step  
        }
        window_df = data.frame(matrix(ncol = b_length+2, nrow = 0))
        
      }
      i = i + 1
    }
    episodes_list[[ep_index]] = new_ep
  }
  
  return(episodes_list)
}

#Function used for over_sampled the data by trying to equalize the data of the 2 different classes in each episode
#separately
#(TRUE class: the target event will happen to the given time period)
#(FALSE class: the target event will not happen to the given time period)
my_mil <- function(milw,episodes_list,b_length,N,threas=0.7){
  if(!csv){
    print("~~~~~~~APPLYING PROPREPROCESSING: MULTI INSTANCE LEARNING~~~~~~~")
  }
  window_df = data.frame(matrix(ncol = b_length+2, nrow = 0))
  
  colnames(window_df) <- colnames(episodes_list)
  for(ep_index in (1:length(episodes_list))){
    
    ep = episodes_list[[ep_index]]
    
    #if(nrow(ep)<milw){
    #next
    #}
    
    days_of_the_episode=(length(ep[,1]))
    if(ep_index>0){
      
      positive_days=length(which(ep$Risk_F>threas))
      negative_days=length(which(ep$Risk_F<=threas))
      
      if(negative_days==0||positive_days==0)
      {
        
        milw_pos=-1
        milw_neg=-1
      }
      else{
        if(positive_days>negative_days){
          if(positive_days>=2*negative_days){
            milw_neg=2
            step_neg=1
            milw_pos=-1
            step_pos=-1
            
          }
          else if(positive_days>1.5*negative_days){
            milw_neg=2
            step_neg=1
            days_to_make=2*negative_days-1-positive_days
            
            milw_pos=positive_days-(days_to_make)
            
            step_pos=1
          }
          else if(positive_days>1*negative_days){
            
            milw_neg=floor(1/4*negative_days)
            
            step_neg=1
            
            days_to_make=negative_days+(negative_days-milw_neg)-positive_days
            
            milw_pos=positive_days-(days_to_make)
            
            step_pos=1
            
          }
        }
        else if(negative_days>positive_days){
          if(negative_days>=2*positive_days){
            milw_neg=-1
            step_neg=-1
            milw_pos=2
            step_pos=1
            
          }
          else if(negative_days>1.5*positive_days){
            
            step_neg=1
            milw_pos=2
            step_pos=1
            
            days_to_make=2*positive_days-1-negative_days
            
            milw_neg=negative_days-(days_to_make)
            
          }
          else if(negative_days>1*positive_days){
            
            step_neg=1
            
            milw_pos=floor(1/4*positive_days)
            
            step_pos=1
            
            days_to_make=positive_days+(positive_days-milw_pos)-negative_days
            
            milw_neg=negative_days-(days_to_make)
            
          }
          
        }
        else{
          milw_neg=2
          step_neg=1
          milw_pos=2
          step_pos=1
        }
      }
    }
    
    new_ep = data.frame(matrix(ncol = b_length+2, nrow = 0))
    window_df = data.frame(matrix(ncol = b_length+2, nrow = 0))
    
    milw=(milw_neg)
    
    F_thres=0
    Z=0
    
    F_thres=-100
    finish=negative_days
    i=1
    count=1
    flag=TRUE
    while(i <= finish){
      
      if((count<=finish)&&(count==i)){
        new_ep = rbind(new_ep,ep[count,])
        count=count+1
      }
      else{
        if(flag==FALSE){
          break
        }
      }
      
      
      if(ep[i,][b_length+2] >= F_thres && nrow(window_df) < milw && milw>0 &&flag){
        window_df = rbind(window_df,ep[i,])
      }
      
      if(nrow(window_df) == milw || i == finish && milw>0&&flag){  
        mean = colMeans(window_df)#mean of each column
        
        mean[mean > 0] = 1#binirize the mean of each column
        
        mf = data.frame(as.list(mean))#make list to frame
        
        mf[1] = ep[i,][1]#timestamp changed from 1 to i or row that are at the currend loop
        mf[b_length+2] = ep[i,][b_length+2]#RISK_F changed from 1 to the risk of the current i 
        
        
        new_ep = rbind(new_ep,mf)
        
        if(nrow(window_df) > 1){
          i = i - (nrow(window_df)-N) #moving step  
          if(i+milw>finish){
            flag=FALSE
          }
        }
        window_df = data.frame(matrix(ncol = b_length+2, nrow = 0))
        
      }
      
      i = i + 1
    }
    
    
    new_ep2 = data.frame(matrix(ncol = b_length+2, nrow = 0))
    window_df = data.frame(matrix(ncol = b_length+2, nrow = 0))
    
    i=negative_days+1
    finish=negative_days+positive_days
    milw=(milw_pos)
    F_thres=-100
    
    count=negative_days+1
    flag=TRUE
    while(i <= finish){
      
      if((count<=finish)&&(count==i)){
        new_ep2 = rbind(new_ep2,ep[count,])
        count=count+1
      }
      else{
        if(flag==FALSE){
          break
        }
      }
      
      
      if(ep[i,][b_length+2] >= F_thres && nrow(window_df) < milw && milw>0&&flag){
        window_df = rbind(window_df,ep[i,])
      }
      
      if(nrow(window_df) == milw || i == finish  && milw>0 &&flag){   
        
        mean = colMeans(window_df)#mean of each column
        
        mean[mean > 0] = 1#binirize the mean of each column
        
        mf = data.frame(as.list(mean))#make list to frame
        
        mf[1] = ep[i,][1]#timestamp changed from 1 to i or row that are at the currend loop
        mf[b_length+2] = ep[i,][b_length+2]#RISK_F changed from 1 to the risk of the current i 
        
        new_ep2 = rbind(new_ep2,mf)
        
        if(nrow(window_df) > 1){
          i = i - (nrow(window_df)-N) #moving step  
          if(i+milw>finish){
            flag=FALSE
          }
          
        }
        window_df = data.frame(matrix(ncol = b_length+2, nrow = 0))
        
      }
      
      i = i + 1
    }
    
    new_ep3 = rbind(new_ep,new_ep2)
    
    episodes_list[[ep_index]] = new_ep3
    
  }
  
  return(order_episodes(episodes_list))
}


### Functions for setting data to the right form

**1) function: make_new_attributes**

**2) function: my_ldply**

**3) function: merging_attributes**

**4) function: make_Risk_F**

In [352]:
make_new_attributes<-function(episodes_list,N,moving_step,BW,b_length,days){
  
  ret_episodes=list()
  index=1
  for(i in 1:length(days)){
      
    end=days[[i]]-BW
    
    old_episode_df=list()
    counter=1
    
    while((index+N-1)<end){
      old_episode_df=append(old_episode_df, list(episodes_list[index:(index+N-1),]), 0)
      counter=counter+1
      index=index+moving_step
    }
    
    if(counter>1){
      ret_episodes=append(ret_episodes, list(old_episode_df))
    }

  }
  return(ret_episodes)
  
}


my_ldply<-function(my_episode){
  ret_list=list()
  for(i in 1:length(my_episode)){
    if(length(my_episode[[i]])>0){
      for(j in 1:length(my_episode[[i]])){
        ret_list=append(ret_list, list(my_episode[[i]][[j]]), 0)
      }
    }
  }
  return(ret_list)
}



merging_attributes<-function(my_episode,N){
  ret_episode=list()
  if(length(my_episode)>0){
    for(i in 1:length(my_episode)){
      if(length(my_episode[[i]])>0){
        test1 = my_episode[[i]][ , !(names(my_episode[[i]]) %in% c("Timestamps"))] #delete column Timestamps
        risk_save=my_episode[[i]][N,]$Risk_F
        test1 = test1[ , !(names(test1) %in% c("Risk_F"))] #delete column Timestamps
        #print("1")
        x <- c(c(paste("e_",c(1:(N*b_length)),sep = "")))
        
        vec <- c(t(test1))
        
        
        names(vec) <- x
        
        vec<-as.data.frame(t(vec))
        
        
        vec$Risk_F<-risk_save
        
        
        
        
        
        ret_episode=append(ret_episode, list(vec), 0)
      }
    }
  }
  return(ret_episode)
  
}


make_Risk_F <- function(train_episodes_list,days,s,midpoint){  
  counter=1  
  for(i in 1:length(days)){
    while(counter<=days[[i]]){              
      train_episodes_list[counter,][["Risk_F"]]=compute_F(s,midpoint,counter,days[[i]])
      counter=counter+1    
    }  
  }
  return(train_episodes_list)
}

## Preprocessing of training set

In [353]:
#Preprocessing for training set.
#milw;midpoint;stepness;N;Z;MIN;MAX;BW
#16;16;0.7;16;8;1;16;0
milw=16
midpoint=16
s=0.7
N=16
moving_step=8
min_warning_interval=1
max_warning_interval = 16
BW = 0


TEST_DATA = FALSE
REMOVE_RARE_EVENTS = argv$rre
REMOVE_FREQUENT_EVENTS = argv$rfe
KEEP_ONLY_FIRST_OCCURENESS = argv$kofe
MULTI_INSTANCE_LEARNING_TEXT = argv$milt #MIL as explained in the text
MULTI_INSTANCE_LEARNING_IMAGE = argv$mili #MIL as presented in the figure
FEATURE_SELECTION = argv$fs

MULTI_INSTANCE_LEARNING_TEXT=FALSE
MULTI_INSTANCE_LEARNING_IMAGE=FALSE

episodes_list <- preprocess(training_set,TEST_DATA,REMOVE_RARE_EVENTS,REMOVE_FREQUENT_EVENTS
                            ,KEEP_ONLY_FIRST_OCCURENESS,MULTI_INSTANCE_LEARNING_TEXT
                            ,MULTI_INSTANCE_LEARNING_IMAGE,FEATURE_SELECTION
                            ,top_features,s,midpoint,b_length
                            ,target_event,target_event_frequency_proportion_rare
                            ,max_event_frequency_proportion_frequent,milw,F_thres)


episodes_list[, "Risk_F"] <- 0
        
days=list()
sum=0
counter=1
for(ep in 1:nrow(episodes_list)){
    if(episodes_list[ep,][[paste("e_",target_event,sep="")]]==1)
        {      
            days[[counter]]<-ep
            sum=sum+days[[counter]][1]
            counter<-counter+1
          }  
         
    }  
episodes_list=make_Risk_F(episodes_list,days,s,midpoint)
lets_try=make_new_attributes(episodes_list,N=N,moving_step =moving_step ,BW=BW,b_length,days)
my_final=my_ldply(lets_try)
my_episodes=merging_attributes(my_final,N=N)
merged_episodes1 = ldply(my_episodes, data.frame)

[1] "The removing rare events(columns) are:"
integer(0)
[1] "------------------------------------------------------------------------------------------------------"
[1] "The removing frequent_events events(columns) are:"
[1]  45 103
[1] "------------------------------------------------------------------------------------------------------"
[1] 9639
[1] "The starting episodes_list is:"
[1] "------------------------------------------------------------------------------------------------------"
[1] "After binirizing:"
[1] "------------------------------------------------------------------------------------------------------"
[1] "After first occurancing preprocess:"
[1] "------------------------------------------------------------------------------------------------------"


## Preprocessing of testing set

In [282]:
#Preprocessing for testing set.

TEST_DATA = TRUE
REMOVE_RARE_EVENTS = FALSE
REMOVE_FREQUENT_EVENTS = FALSE
KEEP_ONLY_FIRST_OCCURENESS = FALSE
MULTI_INSTANCE_LEARNING_TEXT = FALSE #MIL as explained in the text
MULTI_INSTANCE_LEARNING_IMAGE = FALSE #MIL as presented in the figure
FEATURE_SELECTION = FALSE

test_episodes <- preprocess(test_set,TEST_DATA,REMOVE_RARE_EVENTS,REMOVE_FREQUENT_EVENTS,KEEP_ONLY_FIRST_OCCURENESS,MULTI_INSTANCE_LEARNING_TEXT,MULTI_INSTANCE_LEARNING_IMAGE,FEATURE_SELECTION,top_features,s,midpoint,b_length,target_event,target_event_frequency_proportion_rare,max_event_frequency_proportion_frequent,milw,F_thres)
        
days=list()
counter=1
sum=0
for(ep in 1:nrow(test_episodes)){
    if(test_episodes[ep,][[paste("e_",target_event,sep="")]]==1)
    {     
         days[[counter]]<-ep
         sum=sum+days[[counter]][1]
        counter<-counter+1
    }  
          
          
}  
        
test_episodes[, "Risk_F"] <- 0
test_episodes=make_Risk_F(test_episodes,days,s,midpoint)
        

test_episodes_list=make_new_attributes(test_episodes,N=N,moving_step =moving_step ,BW=BW,b_length,days)
        
for(row in 1:length(test_episodes_list)){
    test_episodes_list[[row]]=merging_attributes(test_episodes_list[[row]],N)
    test_episodes_list[[row]]=ldply(test_episodes_list[[row]], data.frame)
}
        
        
for(ep in 0:(length(days)-2)){
    days[[length(days)-ep]]<-days[[length(days)-ep]]-days[[length(days)-ep-1]]
}
        
        
for(ep in test_episodes_list){
    ep = ep[ , !(names(ep) %in% c("Timestamps"))]
}

[1] 5100
[1] "The starting episodes_list is:"
[1] "------------------------------------------------------------------------------------------------------"
[1] "After binirizing:"
[1] "------------------------------------------------------------------------------------------------------"


## Function for Feature Selection
**1) function: top_feature_selection**

**2) function: best_feature_selection**

**3) PCA functions:**
      
   -   **3a) remove_zero_columns**
   
   -   **3a) pca_function**
   
   -   **3a) pca_dimensionality_reduction**

**4) function: feature_reduction**

In [283]:
#Funtion for selectiong the top features using ReliefF.
top_feature_selection <- function(merged_episodes,top_features,seed=0){
  #Feature selection using reliefF
  set.seed(seed)  
  #attrEval function -> https://www.rdocumentation.org/packages/CORElearn/versions/1.53.1/topics/attrEval  
  #ReliefF -> https://medium.com/@yashdagli98/feature-selection-using-relief-algorithms-with-python-example-3c2006e18f83      
  estReliefF <- attrEval(Risk_F ~ ., merged_episodes, estimator="RReliefFexpRank", ReliefIterations=50)
  
  #sort indeces of  estReliefF   
  sorted_indeces = order(estReliefF, decreasing = TRUE)
  
  #keep the the top (top_features) "useful" columns of instances data frame  
  merged_episodes = merged_episodes %>% select(sorted_indeces[1:top_features],ncol(merged_episodes))
  
  return(merged_episodes)
}

In [284]:
#Funtion for selectiong the best feature for the selected algorithm using ReliefF.
best_feature_selection <- function(merged_episodes,test_episodes_list,step,choice=0,acc_thres=0.5){
  
  run = TRUE
  max_F1=0 #variable for keeping the max_F1 score
  max_instances_df = data.frame() #empty data frame named max_instances_df
  
  merged_episodes=((merged_episodes[colSums(merged_episodes) > 0]))
  
  i = length(merged_episodes)-1 #-1 for taking out the label
  
  while(run){
    #Feature selection using reliefF
    
    #attrEval function -> https://www.rdocumentation.org/packages/CORElearn/versions/1.53.1/topics/attrEval  
    #ReliefF -> https://medium.com/@yashdagli98/feature-selection-using-relief-algorithms-with-python-example-3c2006e18f83  
    estReliefF <- attrEval(Risk_F ~ . , merged_episodes, estimator="ReliefFexpRank", ReliefIterations=50)
    
    #sort indeces of  estReliefF 
    sorted_indeces = order(estReliefF, decreasing = TRUE)  
    #print(sorted_indeces)
    #keep the the top i "useful" columns of instances data frame  
    merged_episodes = merged_episodes %>% select(sorted_indeces[1:i],ncol(merged_episodes))
    
    for(j in 1:length(test_episodes_list)){
      test_episodes_list[[j]]=(test_episodes_list[[j]][,names(merged_episodes)])
    } 
    
    
    #if choice==1: find F1 score using RF(function: RFfit)
    if(choice==1){
      my.rf = RFfit(merged_episodes,seed,FALSE)
      F1=eval(test_episodes_list,my.rf,acc_thres)
    }
    #if choice==2: find F1 score using XGBoost(function: XGBoostfit)    
    else if(choice==2){
      my.xgb = XGBoostfit(merged_episodes,seed,FALSE)
      F1=eval(test_episodes_list,my.xgb,acc_thres,TRUE)
    }
    else{
      F1=0
    }
    
    
    #if max F1 score is 0(first iteration)  
    if(max_F1 == 0){
      max_F1 = F1
      max_instances_df = merged_episodes
    } else if(F1>max_F1){ #if F1>=max_F1(which means that with less data we have at least the same F1 score)
      max_F1 = F1 #set as new max F1 score the current F1 score
      max_instances_df = merged_episodes #set new max_instances_df the current instances_df
    }
    i = i - step #-step for taking out the least "useful" columns
    if(i <= 0){
      run = FALSE
    }
  }
  return(max_instances_df)
}

In [285]:
#PCA functions

#PCA functions
#Function for removing the zero columns of an array or a data_frame.
remove_zero_columns <- function(data_array){
  data_array=data.frame(data_array)
  return((data_array[colSums(data_array) > 0]))
}




#Functions tha makes pca to a given array.
pca_function <- function(flat_Xtrain_non_zero_cols){
  
  #for every element of the flat_Xtrain_non_zero_cols subtract the mean of its column.  
  for(i in 1:length(flat_Xtrain_non_zero_cols[1,])){
    flat_Xtrain_non_zero_cols[,i]=flat_Xtrain_non_zero_cols[,i]-mean(flat_Xtrain_non_zero_cols[,i])
  }
  
  #Principal Components Analysis  
  pca.out <- prcomp((flat_Xtrain_non_zero_cols),center = TRUE)
  
  #the matrix of variable loadings (i.e., a matrix whose columns contain the eigenvectors)    
  u=(pca.out$rotation*(+1))
  
  return(u)
}  


#Transforms pca feature reduction to the given dimension.
#If data_made_from=NULL the pca was contucted to the given data,
#else the pca was contucted to the given data_made_from.
pca_dimensionality_reduction<- function(data,dimension,pca_X_train,data_made_from=NULL){
  
  if(dimension>(length(data[1,]))){
    print("ERROR: Dimension must be integer smaller than datas' dimension!!!")
    return(data)
  }
  
  if(is.null(data_made_from)){
    for(i in 1:length(data[1,])){
      data[,i]=data[,i]-mean(data[,i])
    }
  }else{
    data=data.frame(data)
    data=data[,names(data_made_from)]
    for(i in 1:length(data[1,])){
      data[,i]=data[,i]-mean(data_made_from[,i])
    }
  }
  
  #from the matrix whose columns contain the eigenvectors  keep the first X=dimension  
  pca_X_train=array(pca_X_train[,1:dimension],dim =c(length(data[1,]),dimension))
  
  #multiply the data(in which first we subtract the mean valumn at every column) with  pca_X_train  
  red_data=data.matrix(data)%*%data.matrix(pca_X_train)
  return(red_data)
}

In [286]:
#Function fo doing the reduction-selection  
feature_reduction <- function(merged_episodes,test_episodes_list,PCA_REDUCTION=FALSE,top_features=0
                              ,thres=0.5,algo_choice=0,seed=0){
  #if PCA_REDUCTION do pca feature extraction-reduction.  
  if(PCA_REDUCTION){
    
    #save as Label-column the last column   
    Risk_F=merged_episodes[,length(merged_episodes[1,])]
    
    #if is already a data_frame   
    if(is.data.frame(merged_episodes)){
      #keep the non zero columns  
      merged_episodes=merged_episodes[,1:(length(merged_episodes[1,])-1)]                           
      merged_episodes = merged_episodes[, colSums(merged_episodes != 0) > 0]
    }else{
      #keep the non zero columns, by using remove_zero_columns   
      merged_episodes=remove_zero_columns(merged_episodes[,1:(length(merged_episodes[1,])-1)])
    }
    
    #save the data that pca will contucted   
    data_made_from=merged_episodes
    
    #eigenvectors    
    pca_X_train=pca_function(merged_episodes)
    
    #make the dimensionality reduction for training_set  
    merged_episodes=pca_dimensionality_reduction(merged_episodes,top_features,pca_X_train)
    
    #make the dimensionality reduction for testing_set    
    for(i in 1:length(test_episodes_list)){
      test_episodes_list[[i]]=pca_dimensionality_reduction(test_episodes_list[[i]],top_features
                                                           ,pca_X_train,data_made_from)
    }
    
    for(i in 1:length(test_episodes_list)){
      test_episodes_list[[i]]=data.frame(test_episodes_list[[i]])
    }
    
    merged_episodes=data.frame(merged_episodes,Risk_F)
    
  }else{
    if(top_features>0){
      #remove columns with all values equal to zero
      merged_episodes = merged_episodes[, colSums(merged_episodes != 0) > 0]
      
      #keep the top features
      merged_episodes = top_feature_selection(merged_episodes,top_features,seed=seed)
      
      #print("Merged_episodes after feature selection:")
      #print(head(merged_episodes))  
      
      #print("The names of the selected fearures are:")
      #print(names(merged_episodes))  
    }else{
      merged_episodes=best_feature_selection(merged_episodes,test_episodes_list,step,choice=algo_choice
                                             ,acc_thres=thres)
      
    }
    
    #from test_episodes_list keep only the columns that instances_df has   
    for(i in 1:length(test_episodes_list)){
      test_episodes_list[[i]]=(test_episodes_list[[i]][,names(merged_episodes)])
    } 
    
  }
  
  #return the new merged_episodes and test_episodes_list    
  MyList<- list("a"=merged_episodes, "b"=test_episodes_list)
  return(MyList)
}

## Functions for Random Forest 

The segmented data in combination
with the risk quantification values are fed into a **Random Forests** algorithm as
a training set to form a regression problem, which is based on the minimization
of the mean squared error.

**1) function: RFfit**

**2) function: eval**

**3) function: plot_fuct**

In [287]:
#Function that uses Random Forest for predictive the target event(i.e. fault). 
RFfit <- function(train_episodes,seed,plotbool){
  set.seed(seed) #for remaining the random output the same
  
  #Training with randomForest model using instances_df(made from training set)
  #Random Forest R documentation and parameters explanation -> https://www.rdocumentation.org/packages/randomForest/versions/4.6-14/topics/randomForest  
  #Random Forest idea -> https://www.youtube.com/watch?v=loNcrMjYh64  
  my.rf <- randomForest(Risk_F ~ .,data=train_episodes,importance=TRUE, ntree=500) #(default ntree=500) 
  
  #if ploot then print the tree informations      
  if((plotbool==TRUE)){
    result = tryCatch({
      varImpPlot(my.rf) 
    }, error = function(e) {
      print("Error at varImpPlot(my.rf)")
    }) 
    
  }
  
  
  
  #if plootbool then print the tree informations  
  if(plotbool==TRUE)
  {
    print("------------------------------------------------------------------------------------------------------")   
    for(i in 1:2){   
      tryCatch(
        expr = {
          print(getTree(my.rf, i,labelVar=TRUE))
        },
        error = function(e){ 
          
        }
      )  
    } 
    print("------------------------------------------------------------------------------------------------------")   
  }
  
  return(my.rf)      
}     
  
#Function that evaluates the results of RandomForest, XGBoost, PLS, KNN.
eval <- function(test_episodes_list,my_boost,acceptance_threshold=0.5
                 ,USE_XGB=FALSE,USE_PLS=FALSE,USE_KNN=FALSE,IS_MAX =FALSE,path="",N=1,moving_step=1,days=list()){
  false_positives = 0
  true_positives = 0
  false_negatives = 0
  i=0
  counter=0
  
  sum=0
  #for every episode in the episodes_list  
  for(ep in test_episodes_list){
    
    
    if(length(ep)>0){
      counter=counter+1
      
      ep = ep[ , !(names(ep) %in% c("Timestamps"))]
      ep = ep[ , !(names(ep) %in% c("Risk_F"))]
      #print("For episode:")
      #print(ep)
      #print(ep)
      
      #keep the positions of the TRUEs  
      #TRUE if prediction is bigger than the acceptance threshold,else FALSE
      if(USE_XGB){
        Prediction <- predict(my_boost, as.matrix(ep)) #make the prediction of the episode
        pred_indeces = as.numeric(which((Prediction >= acceptance_threshold)==TRUE)) #acceptance_threshold=0.5 which(train_labels %in% c(1))
      }else if(USE_PLS){
        Prediction=(data.matrix(as.matrix(ep))) %*% data.matrix(my_boost$reg.coefs[-1])
        Prediction[,1]=Prediction[,1]+as.numeric(my_boost$reg.coefs[1])
        pred_indeces = as.numeric(which((Prediction >= acceptance_threshold)==TRUE)) #acceptance_threshold=0.5
      }else if(USE_KNN){
        if(i==0){
          before_ep=1
          Prediction=my_boost[before_ep:(before_ep+length(ep[,1])-1)]
          i=10
        }
        else{
          Prediction=my_boost[before_ep:(before_ep+length(ep[,1])-1)]
        }
        before_ep=before_ep+length(ep[,1])
        pred_indeces = as.numeric(which((Prediction >= acceptance_threshold)==TRUE))
        
      }else{
        Prediction <- predict(my_boost, ep) #make the prediction of the episode
        pred_indeces = as.numeric(names(Prediction[Prediction >= acceptance_threshold])) #acceptance_threshold=0.5
      }
      
      
      #print("The pred_indeces before:")
      #print("sum")
      #print(sum)
      temp=ceiling(sum/moving_step) 
      #print("start")
      start=temp*moving_step-sum
      #print(start)
      #ep_legth = length(Prediction)
      ep_legth=days[[counter]]
      #print("ep")
      #print(ep_legth)
      #print("length")
      #print(ep_legth)
      if(start>=0&&counter>1){
        pred_indeces=((pred_indeces-1)*moving_step+start+1)
      }else{
        pred_indeces=((pred_indeces-1)*moving_step+N)
      }
      # print("pred_indeces")
      #print(pred_indeces)
      #print("The pred_indeces:")
      #print(pred_indeces)
      
      #if there is warnings before the max interval from the failure(target event)  
      if(length(pred_indeces[pred_indeces < (ep_legth-(max_warning_interval))]) > 0){
        #increase false positives by the number of these warnings    
        false_positives = false_positives + length(pred_indeces[pred_indeces < (ep_legth-(max_warning_interval))])
      } 
      
      #if there is warnings after the max and before the min interval from the failure(target event)     
      if(length(pred_indeces[pred_indeces >= (ep_legth-(max_warning_interval)) & pred_indeces <= (ep_legth-min_warning_interval)]) > 0){
        true_positives = true_positives + 1 #increase true positives by 1
        #if there is no correct warning    
      } else {
        false_negatives = false_negatives + 1 #increase false negatives by 1
      }
      
      sum=sum+days[[counter]]
      #print("------------------------------------------------------------------------------------------------------")    
    }
  }
  
  
  precision = true_positives/(true_positives+false_positives) #calculate the precision of the model
  if((true_positives+false_positives) == 0){
    precision = 0
  }
  recall = true_positives/length(test_episodes_list) #calculate recall of the model
  
  F1 = 2*((precision*recall)/(precision+recall)) #calculate F1 score of the model
  if((precision+recall) == 0){
    F1 = 0
  }
  
  #prints 
  #if(!csv){
  if(TRUE){  
    cat(paste("dataset:",argv$test,"\ntrue_positives:", true_positives,"\nfalse_positives:", false_positives,"\nfalse_negatives:", false_negatives,"\nprecision:", precision,"\nrecall:", recall,"\nF1:", F1, "\n"))
    if(IS_MAX){
      write(paste("milw:",milw," midpoint:", midpoint," stepness:", s,
                  " MIN:", min_warning_interval," MAX:", max_warning_interval,"\n")
            ,file=path,append=TRUE)
      write(paste("dataset:",path,"\ntrue_positives:", true_positives,"\nfalse_positives:"
                  , false_positives,"\nfalse_negatives:", false_negatives,"\nprecision:"
                  , precision,"\nrecall:", recall,"\nF1:", F1,"\n"),file=path,append=TRUE)
    }  
  } else {
    cat(paste(argv$test,",",true_positives,",",false_positives,",",false_negatives,",",precision,",",recall,",",F1,",",argv$fet,",",argv$tet,",",argv$rre,",",argv$rfe,",",argv$kofe,",",argv$mili,",",argv$milt,",",argv$fs,",",argv$top,",",argv$rer,",",argv$fer,",",argv$seed,",",argv$steepness,",",argv$pthres,",",argv$milw,",",argv$milthres,",",argv$midpoint,",",argv$minwint,",",argv$maxwint,"\n",sep=""))
  }
  
  print("F1 score is:")
  print(F1)
}

#plot function
plot_fuct <- function(test_episodes_list, episode_index, my.rf){
  test_episodes = test_episodes_list[[episode_index]][ , !(names(test_episodes_list[[episode_index]]) %in% c("Timestamps"))]
  Prediction <- predict(my.rf, test_episodes)
  results = data.frame(Risk_F=test_episodes$Risk_F,num_Prediction=as.numeric(Prediction))
  mse = mean((Prediction-test_episodes$Risk_F)^2)
  
  chart =ggplot(results,aes((1:nrow(results)))) +
    # geom_rect(aes(xmin = ceiling(nrow(df_test)/2), xmax = nrow(df_test), ymin = -Inf, ymax = Inf),
    #           fill = "yellow", alpha = 0.003) +
    geom_line(aes(y = Risk_F, colour = "Actual")) +
    geom_line(aes(y = num_Prediction, colour="Predicted")) +
    labs(colour="Lines") +
    xlab("Segments") +
    ylab('Risk (F)') +
    ggtitle("Risk Prediction") + # (RR_KF_2YEARS_PAT08)
    theme(plot.title = element_text(hjust = 0.5)) +
    geom_text(aes(label = paste("MSE=",round(mse,3)), x = 20, y = 1), hjust = -2, vjust = 6, color="black", size=4) #add MSE label
  
  # Disable clip-area so that the MSE is shown in the plot
  gt <- ggplot_gtable(ggplot_build(chart))
  gt$layout$clip[gt$layout$name == "panel"] <- "off"
  grid.draw(gt)
}

## Run Random Forest

In [306]:
#Run Random Forest algorithm and print the results.

#feature selection for Random Forest
merged_episodes_rf=merged_episodes1                    
test_episodes_list_rf=test_episodes_list
threas_RF=0.4

top_features=500          
FEATURE_SELECTION=TRUE
PCA_REDUCTION=FALSE

cat("If you use PCA top_features must be less than:",length(merged_episodes_rf[1,])-2,"\n")

if(FEATURE_SELECTION){
  red_list=feature_reduction(merged_episodes_rf,test_episodes_list_rf,PCA_REDUCTION,top_features,thres=threas_RF,algo_choice=1,seed)
  
  merged_episodes_rf=red_list[1]$a
  test_episodes_list_rf=red_list[2]$b
}  

#Run Random Forest algorithm and print the results.  
my.rf = RFfit(merged_episodes_rf,seed,FALSE)
cat("-------------------------------------------------------------\n")
rf_results=eval(test_episodes_list_rf,my.rf,acceptance_threshold=threas_RF,N=N,moving_step = moving_step,days=days)

If you use PCA top_features must be less than: 2415 
-------------------------------------------------------------
dataset: C:/Users/petsi/Documents/ptyxiakh/testing_my_dataset_150.csv 
true_positives: 14 
false_positives: 10 
false_negatives: 8 
precision: 0.583333333333333 
recall: 0.636363636363636 
F1: 0.608695652173913 
[1] "F1 score is:"
[1] 0.6086957


## Functions for XGBoost

**function: XGBoostfit**

In [307]:
#Function that uses XGBoost for predictive the target event(i.e. fault). 
XGBoostfit <- function(train_episodes,seed,plotbool){
  set.seed(seed)
  
  dtrain <- xgb.DMatrix(data=as.matrix(train_episodes[,1:(length(train_episodes[1,])-1)]),label=as.matrix(train_episodes[,(length(train_episodes[1,]))]))
  my.xgb <- xgb.train(data = dtrain, nthread = 2, eta=0.6, nrounds = 10, objective = "binary:logistic",verbose = 2)
  
  #if plootbool then print the tree informations  
  if(plotbool==TRUE)
  {
    print("------------------------------------------------------------------------------------------------------")
    tryCatch(
      expr = {
        print(xgb.plot.tree(model = my.xgb))
      },
      error = function(e){ 
        print(xgb.dump(my.xgb, with_stats = T))
      }
    )  
    
  } 
  return(my.xgb)
  
}

## Run XGBoost

In [308]:
#Run XGB algorithm and print the results.

#feature selection for XGBoost
merged_episodes_XGB=merged_episodes1
test_episodes_list_XGB=test_episodes_list
threas_XGB=0.4

top_features=70      
FEATURE_SELECTION=TRUE
PCA_REDUCTION=FALSE

cat("If you use PCA top_features must be less than:",length(merged_episodes_XGB[1,])-2,"\n")

if(FEATURE_SELECTION){
  red_list=feature_reduction(merged_episodes_XGB,test_episodes_list_XGB,PCA_REDUCTION,top_features,thres=threas_XGB,algo_choice=2,seed=seed)
  
  merged_episodes_XGB=red_list[1]$a
  test_episodes_list_XGB=red_list[2]$b
}

#Run XGBoost algorithm and print the results.
my.xgb = XGBoostfit(merged_episodes_XGB,seed,FALSE)
cat("-------------------------------------------------------------\n")
xgb_results=eval(test_episodes_list_XGB,my.xgb,threas_XGB,USE_XGB=TRUE,N=N,moving_step = moving_step,days=days)

If you use PCA top_features must be less than: 2415 
-------------------------------------------------------------
dataset: C:/Users/petsi/Documents/ptyxiakh/testing_my_dataset_150.csv 
true_positives: 18 
false_positives: 13 
false_negatives: 4 
precision: 0.580645161290323 
recall: 0.818181818181818 
F1: 0.679245283018868 
[1] "F1 score is:"
[1] 0.6792453


## PLS

In [309]:
#Run PLS regression algorithm and print the results.

#feature selection for PLS
merged_episodes_PLS=merged_episodes1
test_episodes_list_PLS=test_episodes_list
threas_PLS=0.4

top_features=50            
FEATURE_SELECTION=TRUE  
PCA_REDUCTION=FALSE

cat("If you use PCA top_features must be less than:",length(merged_episodes_PLS[1,])-2,"\n")

if(FEATURE_SELECTION){
  red_list=feature_reduction(merged_episodes_PLS,test_episodes_list_PLS,PCA_REDUCTION,top_features,seed)
  
  merged_episodes_PLS=red_list[1]$a
  test_episodes_list_PLS=red_list[2]$b
}else{
  for(i in 1:length(test_episodes_list_LPS)){
    test_episodes_list_PLS[[i]]=(test_episodes_list_PLS[[i]][,names(merged_episodes_PLS)])
  } 
}

num_of_features=length(merged_episodes_PLS[1,])
my.pls = plsreg1(merged_episodes_PLS[,-num_of_features],merged_episodes_PLS[,num_of_features], comps=12, crosval=TRUE)
cat("-------------------------------------------------------------\n")
pls_results=eval(test_episodes_list_PLS,my.pls,acceptance_threshold=threas_PLS,USE_PLS=TRUE,N=N,moving_step = moving_step,days=days)

If you use PCA top_features must be less than: 2415 
-------------------------------------------------------------
dataset: C:/Users/petsi/Documents/ptyxiakh/testing_my_dataset_150.csv 
true_positives: 13 
false_positives: 13 
false_negatives: 9 
precision: 0.5 
recall: 0.590909090909091 
F1: 0.541666666666667 
[1] "F1 score is:"
[1] 0.5416667


## KNN regression

In [312]:
#Run KNN regression algorithm and print the results.


#feature selection for KNN regression
merged_episodes_KNN=merged_episodes1
test_episodes_list_KNN=test_episodes_list
threas_KNN=0.3

top_features=10            
FEATURE_SELECTION=TRUE
PCA_REDUCTION=TRUE

cat("If you use PCA top_features must be less than:",length(merged_episodes_KNN[1,])-2,"\n")

if(FEATURE_SELECTION){
  red_list=feature_reduction(merged_episodes_KNN,test_episodes_list_KNN,PCA_REDUCTION,top_features)
  
  merged_episodes_KNN=red_list[1]$a
  test_episodes_list_KNN=red_list[2]$b
}


merged_testing = ldply(test_episodes_list_KNN, data.frame) #merging all lists of episodes list to one data frame
merged_testing = merged_testing[ , !(names(merged_testing) %in% c("Timestamps"))] #delete column Timestamps
merged_testing = merged_testing[ , !(names(merged_testing) %in% c("Risk_F"))]

K=3
knn.fast<- knn(x=as.matrix(merged_episodes_KNN[ , !(names(merged_episodes_KNN) %in% c("Risk_F"))]) ,xnew=as.matrix(merged_testing) , y=(merged_episodes_KNN$Risk_F), k=K,dist.type = "euclidean", type = "R", method = "average")
prediction_knn_fast=(as.numeric(knn.fast))
cat("-------------------------------------------------------------\n")
KNNfast_results=eval(test_episodes_list_KNN,prediction_knn_fast,threas_KNN,USE_KNN = TRUE,N=N,moving_step = moving_step,days=days) 

If you use PCA top_features must be less than: 2415 
-------------------------------------------------------------
dataset: C:/Users/petsi/Documents/ptyxiakh/testing_my_dataset_150.csv 
true_positives: 17 
false_positives: 22 
false_negatives: 5 
precision: 0.435897435897436 
recall: 0.772727272727273 
F1: 0.557377049180328 
[1] "F1 score is:"
[1] 0.557377


# Neural Networks

   ### 1)LSTM

   ### 2)CNN

   ### 3)MLP

### Functions for the implemantation of the selected NN
**1) function: make_model**

**2) function: fit_model**

**2) function: eval_model**

In [313]:
#Function for creating the NN.
make_model<-function(inputs_shape,USE_LSTM=FALSE,USE_CNN=FALSE,lr=0.01){
  
  hidden_layer1=128
  dense_layer1=128
  
  if(USE_LSTM){
    modeling = keras_model_sequential() %>%   
      layer_lstm(units=hidden_layer1, input_shape=inputs_shape ,return_sequences=FALSE)%>%
      #layer_lstm(units=hidden_layer1/4, input_shape=inputs_shape,return_sequences = TRUE) %>%
      #layer_dropout(0.25)%>%
      #layer_lstm(units=hidden_layer1/2,return_sequences = FALSE) %>%
      
      
      layer_dense(units=1)
    
  }else if(USE_CNN){
    number_of_timestamps=inputs_shape[1] 
    print(number_of_timestamps)  
    print(inputs_shape)
    kern_size=3
    if(number_of_timestamps<3){
      kern_size=number_of_timestamps
    }
    modeling = keras_model_sequential() %>%   
      layer_conv_1d(filters=number_of_timestamps,kernel_size=kern_size,input_shape=inputs_shape,activation ="relu") %>%
      layer_conv_1d(filters=number_of_timestamps,kernel_size=kern_size,activation ="relu") %>%
      #layer_max_pooling_1d(pool_size=2) %>%
      #layer_conv_1d(filters=2*X*M,kernel_size=3,activation ="relu") %>%
      #layer_conv_1d(filters=2*X*M,kernel_size=3,activation ="relu") %>%
      layer_global_average_pooling_1d()%>%
      layer_dropout(0.2)%>%
      layer_dense(units=1)
    
    
    
  }else{
    modeling = keras_model_sequential() %>%   
      
      layer_dense(units=dense_layer1/2, input_shape=inputs_shape, activation = "relu") %>%
      #layer_dropout(0.25)%>%
      layer_dense(units=dense_layer1, activation = "relu") %>%
      layer_dropout(0.25)%>%
      layer_dense(units=dense_layer1/2, activation = "relu") %>%
      layer_dropout(0.25)%>%
      layer_dense(units=1)
    
  }
  
  
  learningrate=lr
  opt = optimizer_adam(lr=learningrate)#, momentum=0.9, nesterov=TRUE)
  modeling %>% compile(loss = 'mse',
                       optimizer = opt,
                       metrics = list("mean_absolute_error") #"mean_absolute_error" 
  )
  
  modeling %>% summary()
  return(modeling)
  
}

#Function for fitting the NN with the training_set.
fit_model<-function(modeling,train_set,train_labels,epoch_size,batchs_size,classes_weights,shuffling = FALSE){ 
  modeling %>% fit(train_set,train_labels, epochs=epoch_size,batch_size=batchs_size,shuffle = shuffling)#,class_weight=classes_weights
  return(modeling)
}

#Function for evaluating the NN with the testing_set(or the predicrions that already have been calculated).
eval_model <- function(test_episodes_list=list(),seed=0,modeling=NULL,acceptance_threshold=0.5
                       ,USE_LSTM=FALSE,USE_CNN=FALSE,number_of_timestamps=1,predictions=list(),IS_MAX=FALSE
                       ,moving_step=1,days=list(),N=N){
 
    set.seed(seed) #for remaining the random output the same
    
    counter=0
    sum=0
    false_positives = 0
    true_positives = 0
    false_negatives = 0                         
    i=0
    #acceptance_threshold=0.7
    if(length(test_episodes_list)==0){
      test_episodes_list=predictions
    }
    #for every episode in the episodes_list  
    for(ep in test_episodes_list){
      i=i+1
      counter=counter+1
      #print(i)
      if(length(predictions)==0){
        ep = ep[ , !(names(ep) %in% c("Timestamps"))]
        ep = ep[ , !(names(ep) %in% c("Risk_F"))]
        
        test_set = data.matrix(ep[,1:length(ep[1,])])
        
        if(USE_LSTM||USE_CNN){
          test_set = array(test_set, dim=c(length(ep[,1]),number_of_timestamps,length(ep[1,])/number_of_timestamps))
        }else{
          test_set = array(test_set, dim=c(length(ep[,1]),length(ep[1,])))
        }
        
        
        Prediction = modeling %>% predict(test_set) #make the prediction of the episode
        ep_legth = length(Prediction)
        pred_indeces=which(Prediction %in% c(Prediction[Prediction >= acceptance_threshold])) 
      }
      else{
        Prediction=predictions[[i]]
        ep_legth = length(Prediction)
        pred_indeces=which(Prediction %in% c(Prediction[Prediction == TRUE])) 
        #print(pred_indeces)
      }
      
      temp=ceiling(sum/moving_step) 
      #print("start")
      start=temp*moving_step-sum
      #print(start)
      #ep_legth = length(Prediction)
      ep_legth=days[[counter]]
      #print("ep")
      #print(ep_legth)
      #print("length")
      #print(ep_legth)
      if(start>=0&&counter>1){
        pred_indeces=((pred_indeces-1)*moving_step+start+1)
      }else{
        pred_indeces=((pred_indeces-1)*moving_step+N)
      }
      
      
      
      #if there is warnings before the max interval from the failure(target event)  
      if(length(pred_indeces[pred_indeces < (ep_legth-(max_warning_interval))]) > 0){
        #increase false positives by the number of these warnings    
        false_positives = false_positives + length(pred_indeces[pred_indeces < (ep_legth-(max_warning_interval))])
      } 
      
      #if there is warnings after the max and before the min interval from the failure(target event)     
      if(length(pred_indeces[pred_indeces >= (ep_legth-(max_warning_interval)) & pred_indeces <= (ep_legth-min_warning_interval)]) > 0){
        true_positives = true_positives + 1 #increase true positives by 1
        #if there is no correct warning    
      } else {
        false_negatives = false_negatives + 1 #increase false negatives by 1
      }
      sum=sum+days[[counter]]
    }
    
    
    precision = true_positives/(true_positives+false_positives) #calculate the precision of the model
    if((true_positives+false_positives) == 0){
      precision = 0
    }
    recall = true_positives/length(test_episodes_list) #calculate recall of the model
    
    F1 = 2*((precision*recall)/(precision+recall)) #calculate F1 score of the model
    if((precision+recall) == 0){
      F1 = 0
    }
    
    #prints 
    if(TRUE){
      cat(paste("dataset:",argv$test,"\ntrue_positives:", true_positives,"\nfalse_positives:", false_positives,"\nfalse_negatives:", false_negatives,"\nprecision:", precision,"\nrecall:", recall,"\nF1:", F1, "\n"))
      if(IS_MAX){
        write(paste("milw:",milw," midpoint:", midpoint," stepness:", s,
                    " MIN:", min_warning_interval," MAX:", max_warning_interval,"\n")
              ,file=path,append=TRUE)
        write(paste("dataset:",path,"\ntrue_positives:", true_positives,"\nfalse_positives:"
                    , false_positives,"\nfalse_negatives:", false_negatives,"\nprecision:"
                    , precision,"\nrecall:", recall,"\nF1:", F1,"\n"),file=path,append=TRUE)
      } 
    } else {
      cat(paste(argv$test,",",true_positives,",",false_positives,",",false_negatives,",",precision,",",recall,",",F1,",",argv$fet,",",argv$tet,",",argv$rre,",",argv$rfe,",",argv$kofe,",",argv$mili,",",argv$milt,",",argv$fs,",",argv$top,",",argv$rer,",",argv$fer,",",argv$seed,",",argv$steepness,",",argv$pthres,",",argv$milw,",",argv$milthres,",",argv$midpoint,",",argv$minwint,",",argv$maxwint,"\n",sep=""))
    }
    
    print("F1 score is:")
    print(F1)
    return(F1)

  
}

### Setting the necessary variables

In [314]:
#default: MLP 
USE_LSTM=FALSE
USE_CNN=FALSE

### Prepare the data

In [323]:
#feature selection for NN
merged_episodes_model=merged_episodes1
test_episodes_list_model=test_episodes_list
threas_MODEL=0.4

top_features=500            
FEATURE_SELECTION=TRUE
PCA_REDUCTION=FALSE

cat("If you use PCA top_features must be less than:",length(merged_episodes_model[1,])-2,"\n")

if(FEATURE_SELECTION){
  red_list=feature_reduction(merged_episodes_model,test_episodes_list_model,PCA_REDUCTION,top_features)
  
  merged_episodes_model=red_list[1]$a
  test_episodes_list_model=red_list[2]$b
}



Ytrain=array(merged_episodes_model$Risk_F,dim=c(length(merged_episodes_model[,1]),1,1))
X=merged_episodes_model[ , !(names(merged_episodes_model) %in% c("Risk_F"))] #delete column Risk_F
Xtrain=data.matrix(X)

xdimtr=length(Xtrain[,1])  
ydimtr=length(Xtrain[1,])

if(USE_LSTM||USE_CNN){
  number_of_timestamps=5  
  Xtrain = array(Xtrain, dim=c(xdimtr,number_of_timestamps,ydimtr/number_of_timestamps))
  inputs_shape=c(number_of_timestamps,ydimtr/number_of_timestamps)
}else{
  Xtrain = array(Xtrain, dim=c(xdimtr,ydimtr))
  inputs_shape=c(ydimtr)
}

If you use PCA top_features must be less than: 2415 


### Make-Fit-Evaluate

In [324]:
modeling=make_model(inputs_shape,USE_LSTM,USE_CNN,lr=0.0005)
modeling=fit_model(modeling,Xtrain,Ytrain,25,4)
if(USE_LSTM||USE_CNN){
    eval_model(test_episodes_list_model,seed,modeling,0.4,USE_LSTM,USE_CNN,number_of_timestamps,N=N,moving_step = moving_step,days=days)
}else{
    eval_model(test_episodes_list_model,seed,modeling,0.4,N=N,moving_step = moving_step,days=days)
}

Model: "sequential_14"
________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
dense_44 (Dense)                    (None, 64)                      32064       
________________________________________________________________________________
dense_45 (Dense)                    (None, 128)                     8320        
________________________________________________________________________________
dropout_24 (Dropout)                (None, 128)                     0           
________________________________________________________________________________
dense_46 (Dense)                    (None, 64)                      8256        
________________________________________________________________________________
dropout_25 (Dropout)                (None, 64)                      0           
________________________________________________________________________________
dense

[1] 0.6122449

## Over-Under sampling

**function: sampling_NN**

In [325]:
sampling<-function(train_set,test_set,train_labels=NULL,times=0,repeats=0
                      ,USE_OVERSAMPLING=FALSE,USE_UNDERSAMPLING=FALSE
                      ,sample_thred=0.5,choice=""){
  
  pp1=which(train_labels >= sample_thred) #positions of data that have Label>=sample_thred(belong to the TRUE class)
  pp0=which(train_labels < sample_thred) #positions of data that have Label>=sample_thred(belong to the FALSE class)
  #print(length(pp1))
  #print(length(pp0))
  
  #If pp1>pp0 change them cause we want pp0 always be the class-Label with the most data  
  if(length(pp1)>length(pp0)){
    temp=pp1
    pp1=pp0
    pp0=temp
  }
  
  #an list to save the results
  sum_of_preds=list()
  xdimtest=length(test_set)  
  for(i in 1:xdimtest){
    sum_of_preds[i]=list()
  }
  flag=FALSE #FALSE if for an episode of test_set no prediction made yet, else TRUE
  
  #for repeats times    
  for(i in 1:repeats){
    if(i==2){
      flag=TRUE
    }
    #Use over_sampling or under_sampling so the number of data of each class to be the same  
    if(USE_OVERSAMPLING){
      pp01=pp0
      pp1=sample(pp1, length(pp0), replace=TRUE)
      pall=sample(c(pp01,pp1), times*length(c(pp01,pp1)), replace=TRUE)
    }
    else if(USE_UNDERSAMPLING){
      pp01=sample(pp0, length(pp1), replace=FALSE)
      pall=sample(c(pp01,pp1), times*length(c(pp01,pp1)), replace=TRUE)
    }
    else{
      return(NULL)
    }
    
    #print((pall))  
    
    train_set_i=train_set[pall,]
    
    return(train_set_i)
    
    train_labels_i=train_labels[pall]
    
    #make the model you chose with the new sampled_data  
    modeling_sample=make_model(inputs_shape,USE_LSTM,USE_CNN,lr=0.001)
    
    #fit the model
    modeling_sample=fit_model(modeling_sample,train_set_i,train_labels_i,20,8)#,shuffling = TRUE)#,list("0"=0.1,"1"=2))
    
    count=0
    
    #for every episode in test_set 
    for(ep in test_set){
      ep = ep[ , !(names(ep) %in% c("Timestamps"))]
      ep = ep[ , !(names(ep) %in% c("Risk_F"))]
      
      if(USE_LSTM||USE_CNN){ 
        number_of_timestamps=inputs_shape[1]
        test_ep= data.matrix(ep[,1:length(ep[1,])])
        test_ep = array(test_ep, dim=c(length(ep[,1]),number_of_timestamps,length(ep[1,])/number_of_timestamps))
      }
      else{
        test_ep= data.matrix(ep[,1:length(ep[1,])])
      }  
      
      
      count=count+1
      
      #make the predictions of the current model   
      Prediction = modeling_sample %>% predict(test_ep)
      
      #add the result of the predictions to the sum_of_preds    
      if(flag){
        sum_of_preds[count]=list(sum_of_preds[[count]]+Prediction)
      }
      else{
        sum_of_preds[count]=list(Prediction)
      }
    }
    
  }
  
  #return  the mean of the sum_of_preds
  for(i in 1:xdimtest){
    sum_of_preds[i]=list(sum_of_preds[[i]]/repeats)
  }
  
  return(sum_of_preds)
  
}

### Run the selected Classifier with over-under sampled training_set

In [326]:
train_labels=array(merged_episodes1$Risk_F,dim=c(length(merged_episodes1[,1]),1,1))


train_sample=sampling(merged_episodes1,test_episodes_list_XGB,train_labels=train_labels,times=1,repeats=0
                                          ,USE_OVERSAMPLING=FALSE,USE_UNDERSAMPLING=TRUE
                                          ,sample_thred=0.5)

my.xgb2 = XGBoostfit(train_sample,seed,FALSE)


cat("-------------------------------------------------------------\n")
threas_XGB2=0.8
xgb_results2=eval(test_episodes_list,my.xgb2,threas_XGB,USE_XGB=TRUE,N=N,moving_step = moving_step,days=days)

-------------------------------------------------------------
dataset: C:/Users/petsi/Documents/ptyxiakh/testing_my_dataset_150.csv 
true_positives: 16 
false_positives: 25 
false_negatives: 6 
precision: 0.390243902439024 
recall: 0.727272727272727 
F1: 0.507936507936508 
[1] "F1 score is:"
[1] 0.5079365


## Combine algorithm predictions

**1) function: return_preds**

**2) function: combine_algos**

In [327]:
#Function that returns the algos_predictions.
#     2 types of predictions:
#   1)If regression_ret=FALSE the predictions for each algo are TRUE-FALSE(0-1), with the help of a threshold. 
#   2)If regression_ret=TRUE the predictions for each algo are decimal numbers.
return_preds<-function(test_episodes_list,fitted_model,USE_NN=FALSE,USE_LSTM=FALSE,USE_CNN=FALSE
                       ,USE_KNN=FALSE,USE_PLS=FALSE,threas=0.5,regression_ret=FALSE,number_of_timestamps=1){
  
  return_Predictions=list()
  count=0
  knn_flag=0
  
  for(ep in test_episodes_list){
    
    ep = ep[ , !(names(ep) %in% c("Timestamps"))]
    ep = ep[ , !(names(ep) %in% c("Risk_F"))]
    test_set = data.matrix(ep[,1:length(ep[1,])])
    
    if(USE_LSTM||USE_CNN){
      test_set = array(test_set, dim=c(length(ep[,1]),number_of_timestamps,length(ep[1,])/number_of_timestamps))  
    }else{
      test_set = array(test_set, dim=c(length(ep[,1]),length(ep[1,])))
    }
    
    if(USE_LSTM||USE_CNN||USE_NN){
      Prediction = fitted_model %>% predict(test_set) #make the prediction of the episode
      if(!regression_ret){
        Prediction=Prediction>threas
      }
      
    }else if(USE_KNN){
      
      if(knn_flag==0){
        before_ep=1
        Prediction=fitted_model[before_ep:(before_ep+length(ep[,1])-1)]
        knn_flag=10
      }
      else{
        Prediction=fitted_model[before_ep:(before_ep+length(ep[,1])-1)]
      }
      before_ep=before_ep+length(ep[,1])
      if(!regression_ret){
        Prediction = (Prediction >= threas)==TRUE
      }
      
      #pred_indeces = as.numeric(which((Prediction >= threas)==TRUE))
      
    }else if(USE_PLS){
        Prediction=(data.matrix(as.matrix(ep))) %*% data.matrix(fitted_model$reg.coefs[-1])
        Prediction[,1]=Prediction[,1]+as.numeric(fitted_model$reg.coefs[1])
        if(!regression_ret){
            Prediction = ((Prediction >= acceptance_threshold)==TRUE) #acceptance_threshold=0.5  
        }    
    }
    else{
      Prediction <- predict(fitted_model, as.matrix(ep)) #make the prediction of the episode
      if(!regression_ret){
        Prediction=Prediction>threas
      }

    }
    
    count=count+1
    return_Predictions[count]=list(Prediction)
    
  }
  
  return(return_Predictions)
  
}

In [328]:
#Function that combines the algos_predictions.
#     2 types of predictions:
#   1)If reg_pred=FALSE the predictions for each algo are TRUE-FALSE(0-1), with the help of a threshold. 
#   2)If reg_pred=TRUE the predictions for each algo are decimal numbers.

#If return_preds=FALSE the function print the results, else returns the combined predictions.

combine_algos<-function(algos_predictions,percent=0.5,return_preds=FALSE
                        ,seed=0,reg_pred=FALSE,reg_threas=0.5,return_reg=FALSE
                       ,N,moving_step,days,weights=list()){
  
 sum_of_algos=list()
  
  num_of_episodes=length(algos_predictions[[1]])  

  for(i in 1:num_of_episodes){
    
    sum_of_algos[i]=list(array(0,dim=c(length(algos_predictions[[1]][[i]]))))
    count_alg=0
    for(algo in algos_predictions){
      count_alg=count_alg+1  
      pred_ep=(algo)[[i]]
        
      for(j in 1:length(pred_ep)){
        if(reg_pred){
          if(length(weights)==0){   
              sum_of_algos[[i]][[j]]=sum_of_algos[[i]][[j]]+pred_ep[[j]]
          }else{
              sum_of_algos[[i]][[j]]=sum_of_algos[[i]][[j]]+pred_ep[[j]]*as.numeric(weights[[count_alg]])
          }
        }else{
          if(pred_ep[[j]]==TRUE){
               
            if(length(weights)==0){ 
                sum_of_algos[[i]][[j]]=sum_of_algos[[i]][[j]]+1
            }else{
                sum_of_algos[[i]][[j]]=sum_of_algos[[i]][[j]]+as.numeric(weights[[count_alg]])
            }  
              
              
          }
          
        }
      }
    }
  }
  

  if(reg_pred){
    if(return_reg&&return_preds){
        if(length(weights)==0){ 
            for(i in 1:num_of_episodes){
              sum_of_algos[[i]]=sum_of_algos[[i]]/length(algos_predictions)
            }
        }
    }else{
        if(length(weights)==0){ 
            for(i in 1:num_of_episodes){
              sum_of_algos[[i]]=(sum_of_algos[[i]]/length(algos_predictions))>=reg_threas
            }
        }else{
            for(i in 1:num_of_episodes){
              sum_of_algos[[i]]=(sum_of_algos[[i]])>=reg_threas
            }
        }
    }    
  }else{
    for(i in 1:num_of_episodes){
      if(length(weights)==0){   
          sum_of_algos[[i]]=sum_of_algos[[i]]>=ceiling(length(algos_predictions)*percent)
      }else{
          sum_of_algos[[i]]=sum_of_algos[[i]]>=percent   
      }  
      
    }
  }
  
  
  if(return_preds){
    return(sum_of_algos)
  }
  
  eval_model(seed=seed,predictions=sum_of_algos,N=N,moving_step = moving_step,days=days)
}

### Collect the bool-predictions  of each algorithm and print the results

In [329]:
cat("RF results:\n")
cat("-------------------------------------------------------------\n")
bool_preds_RF=return_preds(test_episodes_list_rf,my.rf,threas=threas_RF)
eval_model(seed=seed,predictions=bool_preds_RF,N=N,moving_step = moving_step,days=days)
cat("-------------------------------------------------------------\n")

cat("\nXGB results:\n")
cat("-------------------------------------------------------------\n")
bool_preds_XGB=return_preds(test_episodes_list_XGB,my.xgb,threas=threas_XGB)
eval_model(seed=seed,predictions=bool_preds_XGB,N=N,moving_step = moving_step,days=days)
cat("-------------------------------------------------------------\n")

cat("\nPLS results:\n")
cat("-------------------------------------------------------------\n")
bool_preds_PLS=return_preds(test_episodes_list_PLS,my.pls,USE_PLS=TRUE,threas=threas_PLS)
eval_model(seed=seed,predictions=bool_preds_PLS,N=N,moving_step = moving_step,days=days)
cat("-------------------------------------------------------------\n")

cat("\nKNN results:\n")
cat("-------------------------------------------------------------\n")
bool_preds_KNN=return_preds(test_episodes_list_KNN,prediction_knn_fast,USE_KNN=TRUE,threas=threas_KNN)
eval_model(seed=seed,predictions=bool_preds_KNN,N=N,moving_step = moving_step,days=days)
cat("-------------------------------------------------------------\n")

cat("\nNN results:\n")
cat("-------------------------------------------------------------\n")
bool_preds_NN=return_preds(test_episodes_list_model,modeling,USE_NN=TRUE,USE_LSTM=USE_LSTM,USE_CNN=USE_CNN
                           ,threas=threas_MODEL,number_of_timestamps=inputs_shape[1])
eval_model(seed=seed,predictions=bool_preds_NN,N=N,moving_step = moving_step,days=days)
cat("-------------------------------------------------------------\n")

cat("\nSAMPLE results:\n")
cat("-------------------------------------------------------------\n")
bool_preds_XGB2=return_preds(test_episodes_list,my.xgb2,threas=threas_XGB2)
eval_model(seed=seed,predictions=bool_preds_XGB2,N=N,moving_step = moving_step,days=days)
cat("-------------------------------------------------------------\n")

RF results:
-------------------------------------------------------------
dataset: C:/Users/petsi/Documents/ptyxiakh/testing_my_dataset_150.csv 
true_positives: 14 
false_positives: 10 
false_negatives: 8 
precision: 0.583333333333333 
recall: 0.636363636363636 
F1: 0.608695652173913 
[1] "F1 score is:"
[1] 0.6086957


[1] 0.6086957

-------------------------------------------------------------

XGB results:
-------------------------------------------------------------
dataset: C:/Users/petsi/Documents/ptyxiakh/testing_my_dataset_150.csv 
true_positives: 18 
false_positives: 13 
false_negatives: 4 
precision: 0.580645161290323 
recall: 0.818181818181818 
F1: 0.679245283018868 
[1] "F1 score is:"
[1] 0.6792453


[1] 0.6792453

-------------------------------------------------------------

PLS results:
-------------------------------------------------------------
dataset: C:/Users/petsi/Documents/ptyxiakh/testing_my_dataset_150.csv 
true_positives: 8 
false_positives: 7 
false_negatives: 14 
precision: 0.533333333333333 
recall: 0.363636363636364 
F1: 0.432432432432432 
[1] "F1 score is:"
[1] 0.4324324


...

-------------------------------------------------------------

NN results:
-------------------------------------------------------------
dataset: C:/Users/petsi/Documents/ptyxiakh/testing_my_dataset_150.csv 
true_positives: 15 
false_positives: 12 
false_negatives: 7 
precision: 0.555555555555556 
recall: 0.681818181818182 
F1: 0.612244897959184 
[1] "F1 score is:"
[1] 0.6122449


[1] 0.6122449

-------------------------------------------------------------

SAMPLE results:
-------------------------------------------------------------
dataset: C:/Users/petsi/Documents/ptyxiakh/testing_my_dataset_150.csv 
true_positives: 7 
false_positives: 9 
false_negatives: 15 
precision: 0.4375 
recall: 0.318181818181818 
F1: 0.368421052631579 
[1] "F1 score is:"
[1] 0.3684211


[1] 0.3684211

-------------------------------------------------------------


### Make a list of the algorithms' bool-predictions you want to combine and print the result.

In [330]:
algos_predictions=list(bool_preds_RF,bool_preds_KNN,bool_preds_PLS)

mid=combine_algos(algos_predictions,percent=2/3,seed=seed,return_preds=TRUE)
cat("RF, KNN and PLS combination results:\n")
cat("-------------------------------------------------------------\n")
eval_model(seed=seed,predictions=mid,N=N,moving_step = moving_step,days=days)
cat("-------------------------------------------------------------\n")


cat("\nRF-KNN-PLS and XGBoost combination results:\n")
cat("-------------------------------------------------------------\n")
algos_predictions=list(mid,bool_preds_XGB)
combine_algos(algos_predictions,percent=1/2,seed=seed,N=N,moving_step = moving_step,days=days)
weight=list(0.4,0.6)
combine_algos(algos_predictions,percent=1/2,seed=seed,N=N,moving_step = moving_step,days=days,weight=weight)
cat("-------------------------------------------------------------\n")

RF, KNN and PLS combination results:
-------------------------------------------------------------
dataset: C:/Users/petsi/Documents/ptyxiakh/testing_my_dataset_150.csv 
true_positives: 12 
false_positives: 9 
false_negatives: 10 
precision: 0.571428571428571 
recall: 0.545454545454545 
F1: 0.558139534883721 
[1] "F1 score is:"
[1] 0.5581395


[1] 0.5581395

-------------------------------------------------------------

RF-KNN-PLS and XGBoost combination results:
-------------------------------------------------------------
dataset: C:/Users/petsi/Documents/ptyxiakh/testing_my_dataset_150.csv 
true_positives: 19 
false_positives: 17 
false_negatives: 3 
precision: 0.527777777777778 
recall: 0.863636363636364 
F1: 0.655172413793103 
[1] "F1 score is:"
[1] 0.6551724


[1] 0.6551724

dataset: C:/Users/petsi/Documents/ptyxiakh/testing_my_dataset_150.csv 
true_positives: 18 
false_positives: 13 
false_negatives: 4 
precision: 0.580645161290323 
recall: 0.818181818181818 
F1: 0.679245283018868 
[1] "F1 score is:"
[1] 0.6792453


[1] 0.6792453

-------------------------------------------------------------


### Collect the reg-predictions  of each algorithm and print the results

In [331]:
reg_preds_RF=return_preds(test_episodes_list_rf,my.rf,threas=threas_RF,regression_ret=TRUE)
reg_preds_XGB=return_preds(test_episodes_list_XGB,my.xgb,threas=threas_XGB,regression_ret=TRUE)
reg_preds_PLS=return_preds(test_episodes_list_PLS,my.pls,USE_PLS=TRUE,threas=threas_PLS,regression_ret=TRUE)
reg_preds_KNN=return_preds(test_episodes_list_KNN,prediction_knn_fast,USE_KNN=TRUE,threas=threas_KNN,regression_ret=TRUE)
reg_preds_NN=return_preds(test_episodes_list_model,modeling,USE_NN=TRUE,,USE_LSTM=USE_LSTM,USE_CNN=USE_CNN
                          ,threas=threas_MODEL,regression_ret=TRUE,number_of_timestamps=inputs_shape[1])
reg_preds_SAMPLE=return_preds(test_episodes_list,my.xgb2,threas=threas_XGB2,regression_ret=TRUE)

### Make a list of the algorithms' reg-predictions you want to combine and print the result.

In [333]:
algos_predictions=list(reg_preds_KNN,bool_preds_RF)
reg_threas=0.4
reg_mid1=combine_algos(algos_predictions,seed=seed,return_preds=TRUE,reg_pred=TRUE
                      ,reg_threas=reg_threas,return_reg=FALSE,N=N,moving_step = moving_step,days=days)
cat("KNN and RF combination results:\n")
cat("-------------------------------------------------------------\n")
eval_model(seed=seed,predictions=reg_mid1,N=N,moving_step = moving_step,days=days)
cat("-------------------------------------------------------------\n")

algos_predictions=list(reg_preds_RF,bool_preds_XGB)
reg_threas=0.4
reg_mid2=combine_algos(algos_predictions,seed=seed,return_preds=TRUE,reg_pred=TRUE
                      ,reg_threas=reg_threas,return_reg=FALSE,N=N,moving_step = moving_step,days=days)
cat("RF and XGB combination results:\n")
cat("-------------------------------------------------------------\n")
eval_model(seed=seed,predictions=reg_mid2,N=N,moving_step = moving_step,days=days)
cat("-------------------------------------------------------------\n")


cat("\nKNN-RF, RF-XGBoost and PLS and RF combination results:\n")
cat("-------------------------------------------------------------\n")
algos_predictions=list(reg_mid1,reg_mid2,reg_preds_PLS)
combine_algos(algos_predictions,reg_threas=0.4,seed=seed,reg_pred=TRUE,N=N,moving_step = moving_step,days=days)
cat("-------------------------------------------------------------\n")
combine_algos(algos_predictions,reg_threas=0.4,seed=seed,reg_pred=TRUE,N=N,moving_step = moving_step,days=days
              ,weights=list(0.3,0.3,0.4))

KNN and RF combination results:
-------------------------------------------------------------
dataset: C:/Users/petsi/Documents/ptyxiakh/testing_my_dataset_150.csv 
true_positives: 14 
false_positives: 10 
false_negatives: 8 
precision: 0.583333333333333 
recall: 0.636363636363636 
F1: 0.608695652173913 
[1] "F1 score is:"
[1] 0.6086957


[1] 0.6086957

-------------------------------------------------------------
RF and XGB combination results:
-------------------------------------------------------------
dataset: C:/Users/petsi/Documents/ptyxiakh/testing_my_dataset_150.csv 
true_positives: 18 
false_positives: 13 
false_negatives: 4 
precision: 0.580645161290323 
recall: 0.818181818181818 
F1: 0.679245283018868 
[1] "F1 score is:"
[1] 0.6792453


[1] 0.6792453

-------------------------------------------------------------

KNN-RF, RF-XGBoost and PLS and RF combination results:
-------------------------------------------------------------
dataset: C:/Users/petsi/Documents/ptyxiakh/testing_my_dataset_150.csv 
true_positives: 18 
false_positives: 11 
false_negatives: 4 
precision: 0.620689655172414 
recall: 0.818181818181818 
F1: 0.705882352941177 
[1] "F1 score is:"
[1] 0.7058824


[1] 0.7058824

-------------------------------------------------------------
dataset: C:/Users/petsi/Documents/ptyxiakh/testing_my_dataset_150.csv 
true_positives: 18 
false_positives: 11 
false_negatives: 4 
precision: 0.620689655172414 
recall: 0.818181818181818 
F1: 0.705882352941177 
[1] "F1 score is:"
[1] 0.7058824


[1] 0.7058824